In [1]:
!pip install ultralytics
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.2/663.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.9 MB/s eta 0:00:00


In [2]:
import os
import sys
import yaml
import wandb
import zipfile
from google.colab import drive
from wandb.integration.ultralytics import add_wandb_callback
from ultralytics import YOLO
from ultralytics.engine.trainer import BaseTrainer
from ultralytics.engine.validator import BaseValidator
from ultralytics.utils.torch_utils import get_flops, get_num_params

wandb: WARNING This integration is tested and supported for ultralytics v8.0.186 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.


In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Specify the name of the uploaded zip file
zip_file_name = '/content/drive/MyDrive/Prj_Intro_DL/VOC2012Data.zip'

# Specify the extraction directory
extract_dir = '/content/extracted/'

# Create the extraction directory
!mkdir -p {extract_dir}

# Unzip the file
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [5]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
def on_pretrain_routine_start(trainer: BaseTrainer):
    # initialise classes in the config arguments
    trainer.args.labels = trainer.data['names']
    # Init the run
    wandb.init(
        project=trainer.args.project or "YOLOv8n", name=trainer.args.name, config=dict(trainer.args)
    ) if not wandb.run else wandb.run

def on_pretrain_routine_end(trainer: BaseTrainer):
    paths = trainer.save_dir.glob('*labels*.jpg')
    wandb.run.log({'Labels': [wandb.Image(str(x), caption=x.name) for x in paths]})

def on_train_epoch_end(trainer: BaseTrainer):
    # loss of the training dataset
    wandb.run.log(trainer.label_loss_items(trainer.tloss, prefix="train"), step=trainer.epoch + 1)
    # loss of the validation dataset
    vloss = trainer.validator.loss if hasattr(trainer.validator,'loss') else []
    wandb.run.log(trainer.label_loss_items(vloss, prefix="val"),step=trainer.epoch + 1)


def on_fit_epoch_end(trainer: BaseTrainer):
    wandb.run.log(trainer.metrics, step=trainer.epoch + 1)
    #wandb.run.log(trainer.validator.metrics.results_dict, step=trainer.epoch + 1)

def on_train_end(trainer: BaseTrainer):
    art = wandb.Artifact(type="model", name=f"run_{wandb.run.id}_model")
    if trainer.best.exists():
        art.add_file(trainer.best)
        wandb.run.log_artifact(art)
    files = ['results.png', 'confusion_matrix.png', *(f'{x}_curve.png' for x in ('F1', 'PR', 'P', 'R'))]
    files = [(trainer.save_dir / f) for f in files if (trainer.save_dir / f).exists()]  # filter
    wandb.run.log({'Results': [wandb.Image(str(f), caption=f.name) for f in files]})

def on_params_update(trainer: BaseTrainer, params: dict):
    wandb.run.config.update(params, allow_val_change=True)

def teardown(_trainer: BaseTrainer):
    wandb.finish()

callbacks = (
    {
        "on_pretrain_routine_start": on_pretrain_routine_start,
        "on_pretrain_routine_end": on_pretrain_routine_end,
        "on_train_epoch_end": on_train_epoch_end,
        "on_fit_epoch_end": on_fit_epoch_end,
        "on_train_end": on_train_end,
        "teardown": teardown,
    }
    if wandb
    else {}
)

In [ ]:
# Load a model
model = YOLO('yolov8n.pt')

results = model.train(data='/content/extracted/data.yaml', epochs=20, imgsz=640, batch = 16, save_period = 10, iou = 0.6, dfl = 9, cls = 3, box = 1)
for event, func in callbacks.items():
    model.add_callback(event, func)

100%|██████████| 6.23M/6.23M [00:00<00:00, 127MB/s]


Ultralytics YOLOv8.0.229 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/extracted/data.yaml, epochs=20, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.6, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_

100%|██████████| 755k/755k [00:00<00:00, 22.1MB/s]


Overriding model.yaml nc=80 with nc=20

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

wandb: Currently logged in as: tuandatebayo (yolov8deepproject). Use `wandb login --relogin` to force relogin


...

      20/20      2.43G     0.1166      5.433      6.996          8        640: 100%|██████████| 358/358 [02:03<00:00,  2.90it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 182/182 [01:04<00:00,  2.83it/s]


                   all       5823      15787      0.792      0.656      0.734      0.527

20 epochs completed in 1.065 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.0.229 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3009548 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 182/182 [01:07<00:00,  2.70it/s]


                   all       5823      15787      0.787      0.659      0.734      0.527
             aeroplane       5823        484      0.937        0.7      0.815      0.616
               bicycle       5823        380      0.834      0.705      0.797      0.576
                  bird       5823        629        0.8      0.631      0.716       0.49
                  boat       5823        491      0.707      0.485      0.567       0.33
                bottle       5823        733       0.82      0.477      0.592      0.397
                   bus       5823        320      0.855      0.781      0.837      0.687
                   car       5823       1173      0.824      0.673      0.761       0.55
                   cat       5823        618      0.798      0.833      0.857      0.658
                 chair       5823       1449      0.656      0.518      0.592        0.4
                   cow       5823        347      0.758      0.703      0.731      0.512
           diningtabl

lr/pg0,▃▆███▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
lr/pg1,▃▆███▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
lr/pg2,▃▆███▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
metrics/mAP50(B),▄▁▂▃▄▄▅▅▆▆▆▆▆▇▇█████
metrics/mAP50-95(B),▄▁▁▂▃▃▄▄▅▆▆▆▆▆▇▇▇▇██
metrics/precision(B),▄▁▂▂▄▃▄▄▅▆▆▆▆▆▇█▇▇██
metrics/recall(B),▅▁▂▃▄▅▄▅▅▆▆▆▇▆▇█████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▅▇███▇▇▆▆▅▅▅▄▃▃▃▂▂▁▁


In [7]:
#train more
# Load a model
model = YOLO('yolov8n.pt')

results = model.train(data='/content/extracted/data.yaml', epochs=40, imgsz=640, batch = 16, save_period = 10, iou = 0.6, dfl = 9, cls = 3, box = 1)
for event, func in callbacks.items():
    model.add_callback(event, func)

100%|██████████| 6.23M/6.23M [00:00<00:00, 100MB/s]


Ultralytics YOLOv8.0.230 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/extracted/data.yaml, epochs=40, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.6, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_

100%|██████████| 755k/755k [00:00<00:00, 23.4MB/s]


Overriding model.yaml nc=80 with nc=20

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

wandb: Currently logged in as: tuandatebayo (yolov8deepproject). Use `wandb login --relogin` to force relogin


...

      40/40      2.43G     0.1075      4.147      6.718         12        640: 100%|██████████| 358/358 [02:08<00:00,  2.78it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 182/182 [01:02<00:00,  2.90it/s]


                   all       5823      15787      0.793      0.655      0.732      0.528

40 epochs completed in 2.244 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.0.230 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3009548 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 182/182 [01:08<00:00,  2.65it/s]


                   all       5823      15787      0.796      0.654      0.732      0.529
             aeroplane       5823        484      0.906      0.713      0.804      0.613
               bicycle       5823        380      0.832      0.708      0.779      0.575
                  bird       5823        629      0.836      0.588      0.704      0.478
                  boat       5823        491      0.706      0.456       0.54      0.324
                bottle       5823        733      0.804      0.458      0.571       0.38
                   bus       5823        320      0.874      0.759      0.834      0.684
                   car       5823       1173      0.854      0.648      0.765      0.545
                   cat       5823        618      0.836      0.818      0.882       0.69
                 chair       5823       1449      0.671      0.496      0.581      0.388
                   cow       5823        347      0.734      0.645      0.721      0.512
           diningtabl

lr/pg0,▃▆████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg1,▃▆████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,▃▆████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/mAP50(B),▄▁▂▁▃▂▃▄▄▅▄▅▅▆▅▆▆▆▆▇▇▇▇▇▇▇▇▇██▇▇████████
metrics/mAP50-95(B),▄▁▂▁▃▂▃▃▃▄▄▄▄▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇█▇██████
metrics/precision(B),▄▁▄▃▄▃▄▄▄▅▅▅▆▆▆▆▇▇▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇█▇██▇██
metrics/recall(B),▄▂▂▁▃▃▃▄▄▄▄▅▅▆▅▆▅▆▆▆▆▇▆▇▇▇▇▇█▇▇▇▇▇█▇▇█▇█
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▆▇▇██▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▂▂▂▂▂▁▁▁


#Validating

#Predict

export model

In [ ]:
# Convert weight file to other formats
!yolo export model=./runs/detect/train/weights/best.pt format=onnx

NotImplementedError: ignored

In [ ]:
!cp '/content/runs/detect/train/weights/best.onnx' '/content/drive/MyDrive/Prj_Intro_DL'
!cp '/content/runs/detect/train/weights/best.pt' '/content/drive/MyDrive/Prj_Intro_DL'


NotImplementedError: ignored